# 1. 为什么引入闭包（Closure）？

我们在软件开发中常常需要面对这样的问题：函数需要记住某些上下文（如计数器、配置参数等），但函数默认不具备记忆能力。传统的解决方案是使用全局变量，但是这样容易引发命名冲突和不可控的修改。而闭包的出现，能让函数记住“外部变量”，实现了状态的封装。

假设我们想要创建一个计数器，每次调用时都能递增该计数器。如果没有闭包，我们一般使用一个全局变量来保存计数器的值。

## 1.1 不使用闭包的解决方案

In [2]:
count = 0

def increment():
  global count
  count += 1
  print(f"increment: {count}")

increment()
increment()
increment()

increment: 1
increment: 2
increment: 3


以上方案存在的问题是：
- 使用全局变量可能导致代码难以维护，其他函数如果更改了这个变量，将会导致程序逻辑出现问题；
- 如果我们要创建多个计数器，我们要使用多个全局变量和计数器函数；

## 1.2 使用闭包的解决方案

使用闭包，我们可以创建一个函数，内部维护这个计数器的数值，外部函数返回一个内部函数，这个内部函数可以访问这个外部函数的变量。其中下面代码中的`nolocal`用于在嵌套函数`increment()`中修改外层函数中的变量。如果没有`nolocal`，嵌套函数中的变量会被视为局部变量，无法直接修改外层函数的变量。

In [1]:
def create_counter():
  count = 0

  def increment():
    nonlocal count
    count += 1
    print(f"increment: {count}")

  return increment

c1 = create_counter()
c1()
c1()
c1()
print("*" * 30)

c2 = create_counter()
c2()
c2()
c2()

increment: 1
increment: 2
increment: 3
******************************
increment: 1
increment: 2
increment: 3


### 优势：

- 每个计数器的实例都有自己独立的状态，不会相互干扰；
- 避免了全局变量的使用，更加安全和可靠；

# 2. 闭包详解

## 2.1 闭包概念

闭包是由函数及其引用环境（外层作用域的变量）组成的实体，其核心是跨作用域保留上下文。**闭包（Closure）不是指的内函数本身，而是指内函数和它所引用的外部（非全局）作用域中的变量的组合。**

更准确地说：

- 内函数（Inner Function）：这是指定义在另一个函数（外函数）内部的函数；
- 闭包（Closure）：当内函数被返回到其定义的外函数之外执行时，并且该内函数引用了外函数作用域中的非全局变量，那么这个内函数以及它所“记住”或“捕获”的这些外部变量，共同构成了闭包；
- 内函数如果没有引用外部函数的变量，这些外部函数的变量不会成为闭包的一部分；

**核心思想:** 闭包的强大之处在于，即使外函数已经执行完毕并退出了它的作用域，内函数仍然能够访问并操作那些它所引用的外函数中的变量，这些变量“活着”的时间超过了外函数的生命周期。

特点：

- **变量封装:** 闭包内的外部变量对外不可见，避免污染全局空间；
- **延迟绑定:** 闭包函数执行时动态读取外部变量的最新值（除非外部变量被冻结）；
- **内存管理:** 外部变量被闭包引用时，其生命周期延长至闭包销毁；


### 底层机制

- python 通过`__closure__`属性存储闭包变量（类型为 cell 对象）；
- 查看闭包的内容；

In [22]:
def outer():
  x = 10
  y = 20
  z = 30
  def inner():
    print(x)
    print(y)
  return inner

f = outer()
print(f"闭包一共捕获了：{len(f.__closure__)}个变量")
print(type(f.__closure__[0]), type(f.__closure__[1]))
print(f.__closure__[0].cell_contents)
print(f.__closure__[1].cell_contents)

闭包一共捕获了：2个变量
<class 'cell'> <class 'cell'>
10
20


## 2.2 闭包三要素

闭包的三要素：

1. 函数嵌套定义（外层函数包裹内层函数）；
2. 内层函数引用外层函数的变量；
3. 外层函数返回内层函数；

```python
def 外层函数(参数):
  外部变量 = 值

  def 内层函数(参数):
    引用外部变量
    return 计算结果
  
  return 内层函数
```

## 2.3 基本案例

### 案例1： 预设参数的乘法器

In [28]:
def multiplier_factory(n):
  def multiplier(x):
    return x * n
  return multiplier

multiplier_of_3 = multiplier_factory(3)
multiplier_of_5 = multiplier_factory(5)

# print(len(multiplier_of_3.__closure__))
print(f"1 multiplier_of_3: {multiplier_of_3(1)}")
print(f"2 multiplier_of_3: {multiplier_of_3(2)}")
print(f"3 multiplier_of_3: {multiplier_of_3(1)}")
print(f"1 multiplier_of_5: {multiplier_of_5(1)}")
print(f"2 multiplier_of_5: {multiplier_of_5(2)}")

1 multiplier_of_3: 3
2 multiplier_of_3: 6
3 multiplier_of_3: 3
1 multiplier_of_5: 5
2 multiplier_of_5: 10


### 案例2: 记账软件

在一个记账软件中，通过闭包实现一个“记账”函数，它会记住你的余额，并在每次消费的时候更新。

In [36]:
def create_account(initial_balance):
  balance = initial_balance

  def spend(amount):
    nonlocal balance
    if amount > balance:
      print("余额不足，无法消费！")
    else:
      balance -= amount
      print(f"消费：{amount}元，当前余额：{balance}元")

  def deposit(amount):
    nonlocal balance
    balance += amount
    print(f"存款：{amount}元，当前余额：{balance}元")

  def check_balance():
    print(f"当前余额：{balance}元")

  return {
      "spend": spend,
      "deposit": deposit,
      "check_balance": check_balance
  }

account1 = create_account(1000)
print("Account1:")
account1["spend"](500)
account1["deposit"](200)
account1["check_balance"]()

print("*" * 30)
account2 = create_account(500)
print("Account2:")
account2["spend"](600)
account2["check_balance"]()


Account1:
消费：500元，当前余额：500元
存款：200元，当前余额：700元
当前余额：700元
******************************
Account2:
余额不足，无法消费！
当前余额：500元


## 2.4 应用场景

虽然闭包理解起来比较抽象，但是在实际开发中应用非常广泛：
- **状态保留:** 替代全局变量（如计数器, 游戏角色属性）；
- **函数工厂:** 生成预设参数的函数；
- **回调函数:** GUI事件处理中中保留上下文；

# 3. 装饰器（Decorator）

## 3.1 装饰器又是什么？

闭包的核心时内层函数能够记住外层函数的变量，这种能力让函数具备了“状态保留”的特性。那么，如果我们让闭包“记住”另一个函数，并对其进行加工，会发生什么？

In [44]:
def log(func):
  def wrapper():
    print(f"开始执行：{func.__name__}")
    func()
    print("执行结束！")
  return wrapper

def say_hello():
  print("Hello!")

# 用闭包包裹函数
wrapped_hello = log(say_hello)
wrapped_hello()


开始执行：say_hello
Hello!
执行结束！


在以上的例子中，wrapper是一个闭包，它记住了外层传入的 func，并在调用时添加了日志功能。

为了让代码更加简洁，python 提供了 @ 语法糖，将闭包包裹过程简化为一行代码。

In [50]:
def log(func):
  def wrapper():
    print(f"开始执行：{func.__name__}")
    func()
    print("执行结束！")
  return wrapper

@log    # 语法糖等价于 say_hello = log(say_hello)
def say_hello():
  print("Hello!")

# 用闭包包裹函数
say_hello()

开始执行：say_hello
Hello!
执行结束！


在这个例子中，我们可以体会到装饰器的特点：
- 闭包（wrapper）保留了原函数（func）的引用；
- @log 语法糖自动完成闭包包裹的过程；

## 3.2 装饰器详解

装饰器是一种语法糖，其本质是接受函数作为参数的高阶函数，用于动态修改或者增强函数的行为，遵循开放-封闭原则（对扩展开、对修改封闭）。

特点：
- **非侵入式:** 不修改原函数代码和调用方式；
- **可堆叠:** 多个装饰器可叠加使用；
- **动态性:** 装饰器行为在函数定义时即时生效；

**装饰器的本质**

```python
@timer
def computation(n):
    ...
```

它的底层等于：

```python
def computation(n):
    ...
computation = timer(computation)
```

也就是：
- 你定义了一个原始的 computation 函数；
- 然后用 timer(computation) 得到一个新函数 wrapper；
- 最后 用这个 wrapper 替换掉了原来的 computation；

## 3.3 具体语法

基础形式：
```python
def decorator(func):
  def wrapper(*args, **kwargs):
    # 前置的增强逻辑

    result = func(*args, **kwargs)
    
    # 后置的增强逻辑
    return result
  return wrapper

@decorator
def func():
  # 函数体
```

保留元信息：

使用`functiontools.wraps`避免函数名等元数据丢失。
```python
from functools import wraps

def decorator(func):
  @wraps(func)
  def wrapper(*args, **kwargs):
    return func(*args, **kwargs)
  return wrapper
```

参数化装饰器：
当装饰器需要接收参数时，需通过三层闭包实现。
```python
def repeat(n):
  def decorator(func):               # 外层闭包记住参数n
    def wrapper(*args, **kwargs):    # 内层闭包记住func
      for _ in range(n):
        func(*args, **kwargs)
      return wrapper
    return decorator

```

## 3.4 具体案例

以下通过几个具体的案例，来看看装饰器的用法。

### 案例1：函数耗时统计器，自动测量函数执行的时间，用于性能分析。

In [70]:
import time
from functools import wraps

def timer(func):
  # @wraps(func)
  def wrapper(*args, **kwargs):
    start = time.perf_counter()
    result = func(*args, **kwargs)
    elapsed = time.perf_counter() - start
    print(f"{func.__name__}耗时：{elapsed:.4f}秒")
    return result
  return wrapper

@timer
def computation(n):
  return sum(i*i for i in range(n))

computation(10**7)
print(f"{computation.__name__}")

computation耗时：0.9080秒
wrapper
